# Formulas used in Fractalaxy

 [Fractalaxy](https://github.com/bfxdev/fractalaxy) relies on vector and complex number transformations. This Jupyter notebook shows the underlying concepts and explains where the formulas come from.

## Map Controller 2D

`MapController2D` is a GDScript attached to the main rendering node. It receives `InputEvent` signals in its `_input` function and maintain values of exported variables used by the renderer. This script is generic and can be used for any 2D object that can be panned, zoomed or rotated.

The following variables form an [affine coordinate system](https://encyclopediaofmath.org/wiki/Affine_coordinate_system) of the rendered location on the Map. This can be seen as an **orthogonal basis of the reference frame in the Map**. The variables are `Vector2` objects and can be defined as following:

 - $\vec O$ or `Origin`: coordinates in Map of the point on the top left of the rendered area
 - $\vec X$ or `HorizontalBasis`: vector in map coordinates to go from one rendered pixel to the neighbor pixel to the right
 - $\vec Y$ or `VerticalBasis`: vector in map coordinates to go from one rendered pixel to the neighbor pixel down

The definitions above are given with rendering in mind, and could be probably stated more rigorously. The 3 vectors are sufficient to define any observer on the Map, after transformations such as pan, zoom or rotation. The basis vectors will be kept orthogonal to avoid skew and their length will be kept equal to preserve aspect ratio.

The following variables are computed from the 3 main basis vectors, and made available as help for rendering:

- $\vec C$ or `Center` : coordinates in Map of the center of the rendered area (`Vector2`)
- $D$ or `Diameter` : diameter of the central disk of the rendered area, guaranteed to be visible after any resize or switch landscape/portrait (`float`)
- $Z$ or `Zoom` : zoom factor, i.e. number of pixels per Map distance unit (`float`)
- $\alpha$ or `Angle` : rotation angle of the basis vectors in Radians (`float`)

Finally, the size of the area is provided as necessary parameter:
- $\vec R$ or `Resolution` : size in pixels of the rendered area (`Vector2`)

### Basic change of reference frame

In the rendering code, it is necessary to perform transformations between the "screen coordinates" (rendered area in pixels starting at the top left corner) and "Map coordinates".

Typically in a shader it is necessary to compute the Map coordinates of point $\vec M$ on  the Map, from the screen coordinates $(x,y)$:

$$\begin{equation*} \vec M = \vec O + x\vec X + y\vec Y \tag{1} \end{equation*}$$

This equation is a classical change of reference frame. The start reference frame is the rendered area on the screen, with positive values for $x$ and $y$ (even if the vertical axis points to the bottom). The final reference frame is the area observed on the Map (like a camera), delimited by a rectangular window, image of the rendered area through the transformation.

It can be used to directly determine the `Center` of the rendered area:

$$\vec C = \vec O + \frac{R_x}{2}\vec X + \frac{R_y}{2}\vec Y$$


### Principle for a naturally feeling user interface

Any kind of movement of the observer will change the values of the vectors of the orthogonal basis. Basically, panning the Map means changing the `Origin` in the opposite direction as the movement of the observer. Zooming and rotating will involve a change of the 3 variables in most cases (i.e. the `Origin` will not change only if it is at the coordinates as the center of the transformation). 

The user interacts using the mouse, a touch screen or a keyboard to change its view on the map. Touch and mouse gestures will affect the movements of the observer on the map following this general principle: **during observer movements, the touch points remain invariant in Map coordinates**. As a result, it gives the very natural feeling that the Map just follows the fingers or mouse pointer.

### Pan with one touch point

Panning is the simplest movement, based on a single touch point. The movement is defined between two successive rendered frames (images rendered by the engine):

- Frame $N$ : At this frame the current values of $\vec O$, $\vec X$ and $\vec Y$ are known, and the single touch point $\vec P$ is active
- Frame $N+1$ : New values of $\vec O'$, $\vec X'$ and $\vec Y'$ are the unknowns to be determined, and the single touch point ist still active as $\vec P' = \vec P + \vec \Delta$

An arbitrary point in two successive rendered frames has the Map coordinates, according to equation 1:
$$\begin{align*}\vec{M} & = \vec{O} + P_{x}\vec{X} + P_{y}\vec{Y} \newline
\vec{M'} & = \vec{O'} + (P_x+\Delta_x)\vec{X'} + (P_y+\Delta_y)\vec{Y'}\end{align*}$$

Now following the general principle stated above, the touch point $\vec M$ remains invariant after transformation:
$$\vec{M'} = \vec{M}$$

After replacement:
$$\vec{O'} + (P_x+\Delta_x)\vec{X'} + (P_y+\Delta_y)\vec{Y'} = \vec{O} + P_{x}\vec{X} + P_{y}\vec{Y} \tag{2}$$

At this point, because it is a pan operation, it can be assumed that $\vec X' = \vec X$ and $\vec Y' = \vec Y$ (pure translation). Consequently the equation can be simplified to:

$$\vec{O'} = \vec{O} - \Delta_x\vec{X} - \Delta_y\vec{Y}$$

### Zoom by a factor around one touch point

A simple way to zoom is to multiply the basis vectors $\vec X$ and $\vec Y$ by the same factor $a$. Such a simple zoom is centered on the Origin, which is not the most convenient for the user.Now if the mouse wheel is used to zoom, the mouse pointer can act as center of the transformation, the Map would be zoomed around it.

The equation 2 can be re-used, with the difference that the touch point is not moving, i.e. $\vec \Delta = \vec 0$:

$$\begin{align*}\vec{O'} + P_x\vec{X'} + P_y\vec{Y'} & = \vec{O} + P_{x}\vec{X} + P_{y}\vec{Y}\end{align*}$$

The zoom factor $a$ is introduced and the main equation can be simplified:

$$\begin{align*}
\vec X' & = a\vec X \newline
\vec Y' & = a\vec Y \newline
\vec{O'} & = \vec{O} + (1-a)P_{x}\vec{X} + (1-a)P_{y}\vec{Y}
\end{align*}$$

### Zoom and rotation with 2 touch points

The 3 parameters of the basis are 2-components vectors i.e. the basis has 6 degrees of freedom. With 2 points of 2 components, only 4 degrees of freedom can be changed. To close the gap, the additional constraint of orthogonality of the basis needs to be taken into account. The vector $\vec{X'}$ will be the one to determine, and $\vec{Y'}$ computed as a vector perpendicular to $\vec{X'}$ in the anti-clockwise direction, and with the same size:

$$\begin{align*}
\vec{Y'} &= \begin{bmatrix} -X'_{y} \\ X'_{x} \\ \end{bmatrix}
\end{align*}$$

These equations reflect the transformation of the 2 touch points according to equation 1, and combined as the points are invariant in Map:

$$\vec{O'} + P'_{1x}\vec{X'} + P'_{1y}\vec{Y'} = \vec{O} + P_{1x}\vec{X} + P_{1y}\vec{Y}\tag{3}$$
$$\vec{O'} + P'_{2x}\vec{X'} + P'_{2y}\vec{Y'} = \vec{O} + P_{2x}\vec{X} + P_{2y}\vec{Y}\tag{4}$$

The next step is to eliminate $\vec{O'}$ by subtracting both equations (eliminating $\vec{O}$ as well), $(3)-(4)$ is:

$$(P'_{1x}-P'_{2x})\vec{X'} + (P'_{1y}-P'_{2y})\vec{Y'} = (P_{1x}-P_{2x})\vec{X} + (P_{1y}-P_{2y})\vec{Y}$$




Let's name the intermediate constant values:

$$\begin{align*}
\vec{A} &= \vec{P'_1} - \vec{P'_2}\\
\vec{B} &= (P_{1x}-P_{2x})\vec{X} + (P_{1y}+P_{2y})\vec{Y}
\end{align*}$$

It comes (there is probably a more direct way to come to this relation):

$$A_{x}\vec{X'} + A_{y}\vec{Y'} = \vec B$$

At this point the vector components can be written and re-ordered, and $\vec{Y'}$ can be eliminated using its orthogonality to $\vec{X'}$:

$$\begin{align*}
A_{x}X'_{x} - A_{y}X'_{y} & = B_{x} \\
A_{y}X'_{x} + A_{x}X'_{y} & = B_{y}
\end{align*}$$

This is a classical system of linear equations with 2 variables, the solution is [well-known](https://en.wikipedia.org/wiki/Cramer%27s_rule). There is a solution only if the determinant of the coefficients is not null:

$$D = \begin{vmatrix}
A_x & -A_y \\ 
A_y & A_x
\end{vmatrix} = A_x^2 + A_y^2$$

The determinant is clearly positive, and may be null only if both $A_x$ and $A_y$ are null, i.e. if the touch points are exactly on the same location in the $N+1$ frame. This is assumed not possible as it would mean an infinite zoom out.

The solution is then:

$$\begin{align*}
X'_x & = \frac{A_{x}B_{x} + A_{y}B_{y}}{D} \\
X'_y & = \frac{A_{x}B_{y} - A_{y}B_{x}}{D} \\
Y'_x & = -X'_y \\
Y'_y & = X'_x \\
\vec{O'} & = \vec{O} + P_{1x}\vec{X} + P_{1y}\vec{Y} - P'_{1x}\vec{X'} - P'_{1y}\vec{Y'}
\end{align*}$$
